# Resource Utilization - A Predictive Analytics Story

In [263]:
#Module importing and data setup
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import dash
import dash_core_components as dcc
import dash_html_components as html

csv = pd.read_csv('azurerr.csv')
data = csv.copy()

### Define Classes/Functions

In [231]:
#Prints all unique values in each column of the dataset
#df input = the dataset
def unicol(dataframe):
    columns = list(dataframe)
    for col in columns:
        print(col,'\n',dataframe[col].unique(), '\n')
        
def rolling_hours(data):
    data = data.set_index("created").sort_index()
    hours = data.rolling("360D")["hours"].sum()
    data["addhours"] = hours
    return data

In [225]:
#Return unique values in each row
unicol(data)

Resource Request Id 
 ['RR-0040907' 'RR-0042610' 'RR-0042638' 'RR-0043407' 'RR-0043408'
 'RR-0043447' 'RR-0043449' 'RR-0043698' 'RR-0043700' 'RR-0043957'
 'RR-0043959' 'RR-0043967' 'RR-0042801' 'RR-0043751' 'RR-0043950'
 'RR-0043958' 'RR-0043960' 'RR-0044064' 'RR-0044427' 'RR-0042802'
 'RR-0044421' 'RR-0044542' 'RR-0043912' 'RR-0044420' 'RR-0044704'
 'RR-0043627' 'RR-0044285' 'RR-0044422' 'RR-0044812' 'RR-0042611'
 'RR-0043336' 'RR-0043448' 'RR-0043450' 'RR-0043699' 'RR-0043701'
 'RR-0043913' 'RR-0043951' 'RR-0044242' 'RR-0044243' 'RR-0044287'
 'RR-0044423' 'RR-0044541' 'RR-0043693' 'RR-0043696' 'RR-0044424'
 'RR-0044993' 'RR-0044994' 'RR-0045505' 'RR-0045017' 'RR-0045018'
 'RR-0045618' 'RR-0044705' 'RR-0045446' 'RR-0045445' 'RR-0045798'
 'RR-0045023' 'RR-0046671' 'RR-0044288' 'RR-0044425' 'RR-0044540'
 'RR-0045020' 'RR-0045024' 'RR-0045196' 'RR-0045506' 'RR-0045779'
 'RR-0045780' 'RR-0045781' 'RR-0045914' 'RR-0046079' 'RR-0046081'
 'RR-0046145' 'RR-0046146' 'RR-0046185' 'RR-0046547' '

In [226]:
#Create new names for columns that are easier to manage
#Assign new names back to dataframe
data.columns = ['id','priority','status','suggested_resource','project','role','start','end','hours','schedule','held','resource','rate','created','pm']
data_drop = data.drop(columns=['id','suggested_resource','priority','status','schedule','held','rate'])
data_drop.head()

,project,role,start,end,hours,resource,created,pm
0,Sidley - Cloud Strategy and Roadmap - Fixed Price,Senior Solutions Architect: Public Cloud,1/1/2019,1/4/2019,8.0,Blerim Kuliqi,9/14/2018,Pooja Kolla
1,CUNA - AEA Integration Azure T&M Resource - T & M,Senior Solutions Architect: Public Cloud,1/1/2019,1/31/2019,128.0,Zach Harris,11/15/2018,Jim Raymonds
2,CUNA - AEA Compliance Assessment and Remediati...,Engineer: Public Cloud,1/1/2019,1/31/2019,152.0,Brian Buffington,11/16/2018,Jim Raymonds
3,DTE NMS AZURE Buildout - Fixed Price,Senior Solutions Architect: Public Cloud,1/1/2019,1/31/2019,29.0,Arif Mahmood,12/13/2018,Pooja Kolla
4,DTE NMS AZURE Buildout - Fixed Price,Engineer: Public Cloud,1/1/2019,1/31/2019,61.0,Blerim Kuliqi,12/13/2018,Pooja Kolla


In [239]:
#Check columns and types
#Create df copy
data_drop.info()
data_drop_clean = data_drop.copy()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 8 columns):
project     496 non-null object
role        496 non-null object
start       496 non-null object
end         496 non-null object
hours       496 non-null float64
resource    496 non-null object
created     496 non-null object
pm          496 non-null object
dtypes: float64(1), object(7)
memory usage: 31.1+ KB


In [242]:
#Identify columns that should be a datetime type. Add to list
#Update columns to be of datetime type
# datecols = ['start','end','created']
# data_drop_clean[datecols] = data_drop[datecols].apply(pd.to_datetime,errors='coerce')
data_drop_clean['created'] = data_drop['created'].apply(pd.to_datetime,errors='coerce')

In [243]:
#Validating dtype conversions
data_drop_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 8 columns):
project     496 non-null object
role        496 non-null object
start       496 non-null object
end         496 non-null object
hours       496 non-null float64
resource    496 non-null object
created     496 non-null datetime64[ns]
pm          496 non-null object
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 31.1+ KB


In [244]:
#Creating rolling window for cumulative hours by date
grouped_hours = data_drop_clean.groupby('resource').apply(rolling_hours)

In [245]:
zach = grouped_hours.loc['Zach Milleson']

In [246]:
zach

,project,role,start,end,hours,resource,pm,addhours
created,,,,,,,,
2019-02-20,AHEAD - Service Delivery Framework (Cloud),Senior Technical Architect: Automation/Orchest...,2/18/2019,2/22/2019,16.0,Zach Milleson,Ryan Heringhaus,16.0
2019-03-27,DTE Energy Cloudera Web Frontend Deployment - ...,Senior Engineer: Public Cloud,4/22/2019,4/30/2019,28.0,Zach Milleson,Pooja Kolla,44.0
2019-03-28,Medline - Azure Foundational Design - Fixed Price,Engineer: Public Cloud,5/13/2019,6/7/2019,54.0,Zach Milleson,Pooja Kolla,98.0
2019-03-28,Medline - Azure Foundational Design - Fixed Price,Engineer: Public Cloud,6/19/2019,7/19/2019,98.0,Zach Milleson,Pooja Kolla,196.0
2019-04-12,DTE Energy Cloudera Web Frontend Deployment - ...,Senior Engineer: Public Cloud,5/1/2019,5/31/2019,64.0,Zach Milleson,Pooja Kolla,260.0
2019-05-09,Boys Town - CMP - SVCS - vRealize Automation D...,Technical Architect: Automation/Orchestration,5/13/2019,5/31/2019,19.0,Zach Milleson,Jim Raymonds,279.0
2019-05-29,DTE Energy Cloudera Web Frontend Deployment - ...,Senior Engineer: Public Cloud,6/3/2019,6/7/2019,6.0,Zach Milleson,Pooja Kolla,285.0
2019-06-02,Boys Town - CMP - SVCS - vRealize Automation D...,Technical Architect: Automation/Orchestration,6/3/2019,6/7/2019,6.0,Zach Milleson,Jim Raymonds,291.0
2019-07-01,Alix_Azure_Relativity_ Automation - Fixed Price,Senior Solutions Architect: Public Cloud,8/18/2019,10/11/2019,42.0,Zach Milleson,Carri Albrecht,333.0


In [264]:
plot = px.bar(zach, x='resource',y='addhours',animation_frame='start',range_y=[0,2000])

In [ ]:
app = dash.Dash()
app.layout = html.Div([dcc.Graph(figure=plot)])

app.run_server()

Running on http://127.0.0.1:8050/
Running on http://127.0.0.1:8050/
Debugger PIN: 481-898-583
Debugger PIN: 481-898-583
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on
